<a href="https://colab.research.google.com/github/vtsimoes/class_victor_dataset/blob/main/G_BERT_FT_Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ajuste-fino do modelo BERT-base do Bertimbau e Geração da representação computacional dos documento do Dataset VICTOR.

In [ ]:
!pip install simpletransformers

In [ ]:
import pickle
from google.colab import drive
import os
import IPython
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
import matplotlib.cm as cm
import time
%matplotlib inline

In [ ]:
path_dados = '' #path com os dados pré-processados
path_transformados = '' #path para salvar os dados transformados
path_model = '' #path para salvar o modelo com ajuste fino
path_save_model = os.path.join(path_model,'model')
path_results = path_model + 'Resultados/'
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
def save_pickle(data_to_save,path='',filename='file.pkl'):
  open_file = open(os.path.join(path,filename), "wb")
  pickle.dump(data_to_save, open_file)
  open_file.close()

def load_pickle(path='',filename='file.pkl'):
  open_file = open(path + filename, "rb")
  pkl_file = pickle.load(open_file)
  open_file.close()
  return pkl_file

# Finetunning do BERT

In [ ]:
from simpletransformers.language_modeling import (LanguageModelingModel)

## Carregamento e Transformação dos Dados

In [ ]:
doc_train = load_pickle(path = path_dados,filename='train_processed_per_page_sw.pkl')
doc_valid = load_pickle(path = path_dados,filename='valid_processed_per_page_sw.pkl')

In [ ]:
train_file = os.path.join(path_model,'train.txt')
valid_file = os.path.join(path_model,'valid.txt')

In [ ]:
with open(train_file,'w') as file:
  for doc in doc_train['text'].values:
    file.write(doc+'\n')

In [ ]:
with open(valid_file,'w') as file:
  for doc in doc_valid['text'].values:
    file.write(doc+'\n')

## Ajuste fino do modelo

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "block_size": 256,
    "max_seq_length": 256,
    "learning_rate": 2e-5,
    "train_batch_size": 8,
    'eval_batch_size': 8,
    "gradient_accumulation_steps": 8,
    "num_train_epochs": 3,
    "mlm": True,
    "mlm_probability": 0.15,
    "sliding_window": True,
    "stride" : 0.8,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 140000,
    "output_dir": os.path.join(path_model,'model'),
    "save_model_every_epoch": False,
    'hidden_dropout_prob': 0.1,
    'overwrite_output_dir': True
}

In [ ]:
model = LanguageModelingModel("bert", 'neuralmind/bert-base-portuguese-cased', args=train_args,use_cuda=True,cuda_device=0)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

50

In [ ]:
model.train_model(train_file, eval_file=valid_file,show_running_loss=True)

  0%|          | 0/149132 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/31214 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/94672 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/31214 [00:00<?, ?it/s]

  0%|          | 0/94672 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/31214 [00:00<?, ?it/s]

  0%|          | 0/94672 [00:00<?, ?it/s]

(11703,
 {'global_step': [3901, 7802, 11703],
  'perplexity': [tensor(7.3957), tensor(nan), tensor(nan)],
  'eval_loss': [2.0009002205269524, nan, nan],
  'train_loss': [2.04939341545105, 1.9622383117675781, 1.8602898120880127]})

In [ ]:
results_eval = model.eval_model(valid_file)

  0%|          | 0/94672 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/19600 [00:00<?, ?it/s]

{'eval_loss': 1.8119654356753478, 'perplexity': tensor(6.1225)}

# Transformando dados Com BERT

## Usando transformers

In [ ]:
doc_train = load_pickle(path=path_dados,filename='train_processed_concat_pages_sw.pkl')
doc_valid = load_pickle(path=path_dados,filename='valid_processed_concat_pages_sw.pkl')
doc_test = load_pickle(path=path_dados,filename='test_processed_concat_pages_sw.pkl')
seq_len = 512

In [ ]:
max_seq_len = 512
load_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "block_size": 512,
    "max_seq_length": max_seq_len,
    "learning_rate": 2e-6,
    "train_batch_size": 8,
    'eval_batch_size': 8,
    "gradient_accumulation_steps": 8,
    "num_train_epochs": 3,
    "mlm": True,
    "mlm_probability": 0.15,
    "sliding_window": True,
    "stride" : 0.8,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 140000,
    "output_dir": os.path.join(path_model,'model'),
    "save_model_every_epoch": False,
    'hidden_dropout_prob': 0.1,
    'overwrite_output_dir': True
}

In [ ]:
!pip install transformers # Instalação da biblioteca Transformers não padrão do Google Colab

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel

In [ ]:
from transformers.models.bert.modeling_bert import BertConfig
config = BertConfig.from_pretrained(os.path.join(path_model,'model')) #carregamento do modelo com ajuste fino
config

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained(os.path.join(path_model,'model'),model_max_length=512)
model = BertModel.from_pretrained(os.path.join(path_model,'model')).to(device)

Some weights of BertModel were not initialized from the model checkpoint at drive/MyDrive/MestradoTucurui/Experiments/BERT/model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def transform(df,model,tokenizer,seq_len,tipo='cls', divide=10): #type pode ser cls ou mean
  inteiro = len(df)//divide
  resto = len(df)%divide
  i=0 #0-9
  train = []
  while (i < divide):
    print(str(i*inteiro),":",str(inteiro*(i+1)))
    tokenized_train = tokenizer(df["body"][i*inteiro:inteiro*(i+1)].values.tolist(), padding = True, truncation = True, return_tensors="pt")
    tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
  #tokenized_val = tokenizer(doc_valid["body"].values.tolist() , padding = True, truncation = True,  return_tensors="pt")
    with torch.no_grad():
      hidden_train = model(**tokenized_train) #dim : [batch_size(nr_sentences), tokens, emb_dim]
      cls_train = hidden_train.last_hidden_state[:,0,:]
    train.append(cls_train.to("cpu"))
    i = i + 1

  if resto != 0:
    print("Resto")
    print(str(len(df)-resto))
    tokenized_train = tokenizer(df["body"][len(df)-resto:].values.tolist(), padding = True, truncation = True, return_tensors="pt")
    tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}

    with torch.no_grad():
      hidden_train = model(**tokenized_train) #dim : [batch_size(nr_sentences), tokens, emb_dim]
      cls_train = hidden_train.last_hidden_state[:,0,:]

    train.append(cls_train.to("cpu"))

  print(tokenized_train.keys())

#move on device (GPU)

  #tokenized_val = {k:torch.tensor(v).to(device) for k,v in tokenized_val.items()}


  #hidden_val = model(**tokenized_val)

  #get only the [CLS] hidden states

  #cls_val = hidden_val.last_hidden_state[:,0,:]

  labels = df.document_type.values
  files = df.file_name.values
  #return doc_vecs,vecs

  data_dict = {'w2v':train,'labels':labels,'filename':files}
  return data_dict

### Gerando conjunto de Treino

In [ ]:
import gc

In [ ]:
torch.cuda.empty_cache()
gc.collect()

642

In [ ]:
bert_train = transform(doc_train,model,tokenizer,tipo=tipo,seq_len=seq_len,divide=300)
save_pickle(bert_train,path=os.path.join(path_transformados),filename='train_bert_mlm_512_specialized_O100.pkl') #OK

In [ ]:
torch.cuda.empty_cache()
gc.collect()

84

In [ ]:
tipo = 'cls'

In [ ]:
bert_train_NO = transform(doc_train.loc[doc_train.document_type != 'outros'],model,tokenizer,tipo=tipo,seq_len=seq_len,divide=100)
save_pickle(bert_train_NO,path=os.path.join(path_transformados),filename='train_bert_mlm_512_specialized_NO.pkl') #OK

In [ ]:
import pandas as pd
filenames_O10 = load_pickle(path=path_dados,filename='filenames_train_O10.pkl')
doc_train_O10 = doc_train.loc[doc_train.file_name.isin(filenames_O10)]
doc_train_O10 = pd.concat([doc_train_O10,doc_train.loc[doc_train.document_type != 'outros']], ignore_index=True, sort=False)
bert_train_O10 = transform(doc_train_O10,model,tokenizer,tipo=tipo,seq_len=seq_len,divide=150)
save_pickle(bert_train_O10,path=os.path.join(path_transformados),filename='train_bert_mlm_512_specialized_O10.pkl') #OK

### Gerando conjunto de Validação

In [ ]:
bert_valid = transform(doc_valid,model,tokenizer,tipo=tipo,seq_len=seq_len,divide=150)
save_pickle(bert_valid,path=os.path.join(path_transformados),filename='valid_bert_mlm_512_specialized_O100.pkl') #OK

In [ ]:
bert_valid_NO = transform(doc_valid.loc[doc_valid.document_type != 'outros'],model,tokenizer,tipo=tipo,seq_len=seq_len,divide=100)
save_pickle(bert_valid_NO,path=os.path.join(path_transformados),filename='valid_bert_mlm_512_specialized_NO.pkl')

In [ ]:
filenames_O10 = load_pickle(path=path_dados,filename='filenames_valid_O10.pkl')
doc_valid_O10 = doc_valid.loc[doc_valid.file_name.isin(filenames_O10)]
doc_valid_O10 = pd.concat([doc_valid_O10,doc_valid.loc[doc_valid.document_type != 'outros']], ignore_index=True, sort=False)
bert_valid_O10 = transform(doc_valid_O10,model,tokenizer,tipo=tipo,seq_len=seq_len,divide=100)
save_pickle(bert_valid_O10,path=os.path.join(path_transformados),filename='valid_bert_mlm_512_specialized_O10.pkl') #OK

### Gerando Conjunto de Teste

In [ ]:
bert_test = transform(doc_test,model,tokenizer,tipo=tipo,seq_len=seq_len,divide=200)
save_pickle(bert_test ,path=os.path.join(path_transformados),filename='test_bert_mlm_512_specialized_O100.pkl') #OK

In [ ]:
bert_test_NO = transform(doc_test.loc[doc_test.document_type != 'outros'],model,tokenizer,tipo=tipo,seq_len=seq_len,divide=100)
save_pickle(bert_test_NO,path=os.path.join(path_transformados),filename='test_bert_mlm_512_specialized_NO.pkl') #OK